!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
pip install --no-cache-dir --index-url https://pypi.nvidia.com pytorch-quantization !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 ./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
./ch train --config configs/examples/toy_uniform_tensorRT.toml
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

sudo apt update && sudo apt upgrade -y 

wget https://repo.continuum.io/archive/Anaconda3-2023.09-0-Linux-x86_64.sh 

bash Anaconda3-2023.09-0-Linux-x86_64.sh 

Close and reopen terminal

source /root/.bashrc

conda config --set auto_activate_base false

git clone https://github.com/mau-mar/mase/

cd mase

bash scripts/init-conda.sh

source /opt/conda/etc/profile.d/conda.sh

conda activate mase

conda config --add channels conda-forge

git checkout origin/mauro-tensorRT-integration

cuda-python
absl-py
scipy
prettytable
sphinx-glpi-theme

./ch train --config configs/examples/toy_uniform_tensorRT.toml

In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt
import copy
from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

from chop.tools.logger import set_logging_verbosity
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model
from chop.dataset import MaseDataModule, get_dataset_info

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    tensorrt_analysis_pass,
    )
set_logging_verbosity("info")

/opt/conda/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-16 11:20:23,406] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0316 11:20:25.071521 140497372997440 logger.py:44] Set logging level to info


In [2]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
#CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"
# CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-trt_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"
CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"
# CHECKPOINT_PATH = "/root/mase/TensorRTDev/vgg7-test-accu-0.9332.ckpt"

model_name = "jsc-toy"
dataset_name = "jsc"

# model_name = "vgg7"
# dataset_name = "cifar10"

max_epochs = 1
batch_size = 256
learning_rate = 1e-3
accelerator = "gpu"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

INFO     Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt
I0316 11:20:31.062528 140497372997440 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt


In [3]:
import toml

# Path to your TOML file
toml_file_path = '/root/mase/machop/configs/examples/jsc_trt_quantization.toml'
# toml_file_path = '/root/mase/machop/configs/examples/vgg7_quantization.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt_quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt_quantize', {})
# Extract the 'passes.tensorrt_fine_tune' section and its children
tensorrt_train_config = pass_args.get('passes', {}).get('tensorrt_fine_tune', {})
# Extract the 'passes.tensorrt_analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt_analysis', {})

# Print or return the extracted section
print("Pass Args:", pass_args)
print("TensorRT Quantize Config:", tensorrt_quantize_config)
print("TensorRT Analysis Config:", tensorrt_analysis_config)

Pass Args: {'model': 'jsc-toy', 'dataset': 'jsc', 'max_epochs': 10, 'batch_size': 256, 'learning_rate': 0.001, 'accelerator': 'gpu', 'passes': {'tensorrt_quantize': {'by': 'type', 'calibrator': ['percentile', 'mse', 'entropy'], 'num_calibration_batches': 10, 'percentiles': [99], 'report': True, 'default': {'config': {'quantize': True, 'calibrator': 'histogram', 'precision': 'FP16', 'input': {'calibrator': 'histogram', 'quantize_axis': False}, 'weight': {'calibrator': 'histogram', 'quantize_axis': False}}}, 'linear': {'config': {'calibrator': 'histogram', 'quantize': True, 'precision': 'FP16'}, 'input': {'calibrator': 'histogram', 'quantize_axis': False}, 'weight': {'calibrator': 'histogram', 'quantize_axis': False}}, 'fine_tune': {'epochs': 1}}, 'tensorrt_analysis': {'num_batches': 500, 'num_GPU_warmup_batches': 5, 'test': True}}}
TensorRT Quantize Config: {'by': 'type', 'calibrator': ['percentile', 'mse', 'entropy'], 'num_calibration_batches': 10, 'percentiles': [99], 'report': True, 

In [4]:
configs = [tensorrt_quantize_config, tensorrt_train_config, tensorrt_analysis_config]
for config in configs:
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

In [5]:
'''
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-trt_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/vgg7-test-accu-0.9332.ckpt --load-type pl
'''
# Restart kernel and re-run previous cells to avoid any issues
# mg_og = deepcopy_mase_graph(mg)
mg_original = copy.deepcopy(mg.model)

# Calibrate model (passing data samples to the quantizer and deciding the best amax for activations)
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)

mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Conduct QAT
# mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Convert and store to ONNX and then TensorRT
mg, trt_meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Applying fake quantization to PyTorch model...
I0316 11:20:32.720956 140497372997440 utils.py:121] Applying fake quantization to PyTorch model...


INFO     Fake quantization applied to PyTorch model.
I0316 11:20:34.096790 140497372997440 utils.py:137] Fake quantization applied to PyTorch model.
INFO     Starting calibration of the model in PyTorch...
I0316 11:20:34.099369 140497372997440 calibrate.py:63] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0316 11:20:34.104685 140497372997440 calibrate.py:73] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 11:20:34.106192 140497372997440 calibrate.py:73] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 11:20:34.107707 140497372997440 calibrate.py:73] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 11:20:34.109252 140497372997440 calibrate.py:73] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibr

[03/16/2024-11:21:47] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO     TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_1/model.trt
I0316 11:23:44.385425 140497372997440 quantize.py:124] TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_1/model.trt
INFO     TensorRT Model Summary Exported to /root/mase/mase_output/TensorRT/Quantization/JSON/2024_03_16/version_1/model.json
I0316 11:23:44.693626 140497372997440 quantize.py:168] TensorRT Model Summary Exported to /root/mase/mase_output/TensorRT/Quantization/JSON/2024_03_16/version_1/model.json


In [8]:
# Compare original tensorrt with quantized graph
trt_meta = {}
from pathlib import PosixPath

# JSC-Toy INT8 only quantization 
trt_meta['graph_path'] = PosixPath('/root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_0/model.trt')

_, _ = tensorrt_analysis_pass(trt_meta['graph_path'], pass_args=tensorrt_analysis_config)

_, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

INFO     
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
I0316 11:19:28.639614 140089548523328 analysis.py:128] 
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
INFO     Starting TensorRT transformation analysis
I0316 11:19:28.641288 140089548523328 analysis.py:202] Starting TensorRT transformation analysis


[03/16/2024-11:19:28] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO     
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73309    |
|      Average Precision       |    0.74935    |
|        Average Recall        |    0.73329    |
|       Average F1 Score       |    0.73688    |
|         Average Loss         |    0.7531     |
|       Average Latency        |  0.18148 ms   |
|   Average GPU Power Usage    |   64.879 W    |
| Inference Energy Consumption | 0.0032707 mWh |
+------------------------------+---------------+
I0316 11:19:32.303160 140089548523328 analysis.py:306] 
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73309    |
|      Average Precision       |    0.74935    |
|        Average Recall        |    0.73329    

In [6]:
# Compare original tensorrt with quantized graph
trt_meta = {}
from pathlib import PosixPath

# JSC-Toy FP16 only quantization 
trt_meta['graph_path'] = PosixPath('/root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_1/model.trt')

_, _ = tensorrt_analysis_pass(trt_meta['graph_path'], pass_args=tensorrt_analysis_config)

_, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

INFO     
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
I0316 11:24:17.414859 140497372997440 analysis.py:128] 
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
INFO     Starting TensorRT transformation analysis
I0316 11:24:17.417420 140497372997440 analysis.py:202] Starting TensorRT transformation analysis


[03/16/2024-11:24:17] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO     
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73325    |
|      Average Precision       |    0.74924    |
|        Average Recall        |    0.73356    |
|       Average F1 Score       |    0.73732    |
|         Average Loss         |    0.75403    |
|       Average Latency        |   0.2609 ms   |
|   Average GPU Power Usage    |   68.244 W    |
| Inference Energy Consumption | 0.0049457 mWh |
+------------------------------+---------------+
I0316 11:24:23.903356 140497372997440 analysis.py:306] 
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73325    |
|      Average Precision       |    0.74924    |
|        Average Recall        |    0.73356    